In [1]:
from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
from scipy import ndimage
import cv2
import pygame

def resizeInput(X, w=200):
    frame = X.astype(np.uint8)
    resized = cv2.resize(frame, dsize=(w, w), interpolation=cv2.INTER_AREA)
    return resized

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def evolution_controller():
    sensors = SensorSet([
        GenomeBinarySensor(genome_id=8, draw=False),
        GenomeBinarySensor(genome_id=9, draw=False)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
        seed=None,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=((1/3) * np.pi), _min=-((2/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
        GeneRule(_max=((2/3) * np.pi), _min=-((1/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
    ]

    phenotype = [
        AverageSpeedBehavior(),
        AngularMomentumBehavior(),
        RadialVarianceBehavior(),
        ScatterBehavior(),
        GroupRotationBehavior(),
        # GeneElementDifference(8, 9)
    ]

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        seed=None,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=15,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=1200,
        display_novelty=True,
        save_archive=True,
        show_gui=True,
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen,
    )

    halted_evolution = ModifiedHaltingEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution, screen

pygame 2.1.2 (SDL 2.0.16, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import numpy as np
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from generation.evolution import ModifiedHaltingEvolution
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import time
from networks.ensemble import Ensemble

trial_name = f"{str(int(time.time()))}"

TRAIN = True
CLUSTER_AND_DISPLAY = True
WRITE_OUT = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ensemble = Ensemble(size=3, output_size=5, lr=30e-4, learning_decay=0.9, decay_step=4, threshold=9.0, weight_decay=1e-6, new_model=True)
ensemble.load_ensemble("dual-mini-hope")
# ensemble.load_ensemble("full-mini-HIL")
network = ensemble.ensemble[2]
network.eval()

SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
EPOCHS = 20
DATA_SIZE = 10000
EVOLUTIONS = 15
K_CLUSTERS = 8

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-mini-dual", rank=0)
evolution, _ = evolution_controller()
dataset = anchor_dataset if EVOLUTION else sampled_dataset
random_archive = DataAggregationArchive(scalar=True)

hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()

for gen in range(EVOLUTIONS):
    # Simulate current population + Save Data
    for i in range(len(evolution.getPopulation())):
        # The collection of the original behavior vector below is only used to collect data to compare with the baseline
        visual_behavior, genome, baseline_behavior = evolution.next()
        dataset.new_entry(visual_behavior, genome, baseline_behavior)

    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

    # Then, evolve
    start_time = time.time()
    embedded_archive = hil.getEmbeddedArchive(dataset, network)
    evolution.overwriteArchive(embedded_archive, random_archive)
    embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
    evolution.overwriteBehavior(embedded_behavior)
    evolution.saveArchive(trial_name)
    evolution.evolve()
    evolution.restart_screen()

    print(f"Evolution complete for gen{gen}")

    # Cluster current dataset, display clusters, and save for analysis
    if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
        hil.embed_and_cluster(network, dataset, auto_quit=True)
        evolution.restart_screen()

HIL Init!
<class 'generation.evolution.ModifiedNoveltyArchieve'>
[  9.60133885  32.18407587  33.68885106  16.9773245    9.88080558
  17.33515328   9.10856378  20.09373025  58.21094199  30.2111022
   9.47994283  24.74974538  24.67943626  62.28253577  39.89208062
  13.7282693   14.65432748  43.4627505   17.57894867  21.59743934
   9.50010025 239.42993772  26.83149844  22.39666334   8.10216335
   9.77933498  12.10056778  17.7569189   17.66899751  25.93146338
  13.89053995 388.46990319   9.0795249   25.61177701  78.61720231
   8.42445605  19.86961483  11.40036805  14.71309995  66.89462217
  59.99012218  13.35907857  10.06240119  17.92144138  15.69732362
   8.81225831   7.92070948   7.8087841   12.93426288  79.72729767
  59.0522809    9.28728347   8.79631791   8.57938218  20.14100463
  25.3599718   83.30117825  19.25922337  13.13063097  57.63741997
  11.25071983   9.52394332  17.81526653  12.2470505   35.51366114
  19.82072708  14.82597162  14.95479075  10.2870984   47.7979501
  20.9343326 

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[45.01852504 19.29783317 21.66134031  8.21596131 33.20866486 13.84036553
 23.58761407 20.50633398 46.0499869  49.40395303 29.19377217 23.51279452
 94.88506076  6.69689695 10.63369188 37.25746452 33.24562131 20.78354378
 29.05930381 23.93635209 25.73037239 10.7007447  47.43182351 13.84267094
  6.85714513 50.20289689 46.7717837   9.87819151 37.03485837 43.89566365
 32.53696642 22.05611323  9.48718487  9.04336497  7.15761876 11.52568597
 22.00351083 20.98445485 27.9899987  25.18742538 22.25849158  9.42626843
 61.14940663 13.90283164 24.68521581  9.49883186 17.42647256  8.45222783
 32.84204452 39.86905299 34.20147278  7.10169591 16.3669844  17.01479743
 40.10400565 60.97340376  8.20307043 16.5566801  29.66821583 19.86906377
 15.72918989 23.52922777 33.12339888 19.28251422 40.29380107 53.31444622
 14.46334376 21.21974606  8.73473063 23.16445035  7.02876134 14.38007598
 22.75392265 20.6703435  13.98595615  8.21322474 44.3395195  44.81785

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 19.19418399   6.5626449    6.03832867  34.51000025  37.11932683
  11.14302261  19.00580821  23.13005638  30.22571261   5.55240342
  25.2091774   61.06657885  12.76770369  25.3086104   20.39386634
  12.76632057  42.45873795  30.73247964  31.77320083  28.73177268
  19.61021351   7.25717859  33.84518219  22.67815259  33.95931646
  25.35884202  30.7331878   30.96978212  44.99467775  31.55378278
  67.17044434   6.28108928  12.34973304  28.47489253  59.10243248
   8.04640197  20.86559938  23.7135334   36.15356117  25.68718237
 109.40194537  26.74714718  17.73938829  81.68965567  11.92270675
  15.2160873   47.89222133  30.37237626  31.96537141  21.62665343
   8.30182098  41.85033385  18.59421762  10.78204985  11.32735328
  47.24786929  16.89377244  26.0882648   63.45418981  10.54907221
  35.08350238  26.3925791   12.13749147  23.98205269  13.14053447
   7.21837844  55.64967966  25.9925084    9.43783492  35.02037778
  12.58824307  27.980

...

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[39.15598177 12.87673517 33.38557357 30.44044746 16.18250216 16.12351059
 26.72362373 14.37069835 23.78980356 29.72154265 19.76631843 29.17254819
  8.55655113 18.25476958 14.34800608 17.85803228 14.49022152 13.13895397
 15.18164197 23.94886605 30.32124259 28.83452052 20.31529156 26.47391323
 18.70097212 25.80272572 17.34074785 26.69261026 27.92890542 22.18489372
  1.4862992  21.94098478 23.05789209 24.2761888  23.36804186 21.24975853
 12.66467203  6.22663744 23.21997882 19.62466131 27.5302957  28.19379692
 29.10904631 29.06803418 17.80010868 21.51440269  8.26254033 24.76663902
  9.38427952 19.70783759 22.20883296 23.82476731 26.21490742 27.9702161
 21.71244344 26.44470043  3.56631193 19.96287478 24.20243728 18.22806694
 16.67051777  4.76639096 15.45628163 27.14130181 23.54260132 15.35722308
 29.32765322 26.0518631  27.98010687 60.10241516 17.33405551 17.4231482
 13.20424759 29.30000947 17.58954107 39.35514683 23.78055463 16.8850372

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[39.02956786 13.54169568 22.63310654 29.58380105 17.75677845 18.53204357
 26.02015601 18.59580653 16.2752085  25.59457377 20.65535313 21.25413787
 21.22176603 25.63161827 22.14943222 20.65050906 56.3874379  21.32635068
 19.0950526  16.04375646 23.95638135 13.42329847 23.2049644  12.86180589
 29.31052044 18.17221079 31.71600402 19.94061001 12.32617772 21.31521118
 25.10187515 16.62562651 25.39198603 14.76852359  7.42607089  2.99339397
 48.55930588 32.57702803  1.76835446 33.81410472 22.13226587 19.75211252
 21.56552019 17.07516797 12.43730807 24.58131989 29.50634209 33.35382381
 20.74746731 21.08551514 13.67177247 21.71664564 19.83327644 21.8442861
 27.84047999 23.33400946 26.15326959 23.25601646 26.34485831 13.84005035
  7.74255946 17.31696986 29.47089846 21.31780581 16.23072323 13.75004726
 26.83041499 23.53734689 19.29962426 16.78738114 17.39953566 21.30921676
 21.33634412 23.12672197 27.61482642 21.8042677  11.14195662 21.346314

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


In [ ]:
from ui.clustering_gui import ClusteringGUI
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()